In [1]:
import pandas

In [4]:
everything = pandas.read_gbq("""select refnr 
	, item_hierarchy 
from `bq-central-dev`.data_migration_products.dbt_products__all_attributes_and_properties
;""")

In [33]:
def fix_node(node):
    sections = node.split(" : ")
    sections = [s.strip().title() for s in sections]
    sections = [s for s in sections if s != ""]
    return " : ".join(sections)


In [34]:
nodes = everything.item_hierarchy.apply(fix_node).unique()

In [35]:
nodes

array(['Fashion : Jewellery : Gold Plated Silver : Necklace : Pendant Necklaces',
       'Fashion : Jewellery : Gold Plated Silver : Bracelet : Charm Bracelets',
       'Fashion : Jewellery : Silver Plated Brass : Bracelet : Chain Bracelets',
       'Fashion : Jewellery : Silver Plated Brass : Anklet : Chain Anklets',
       'Fashion : Jewellery : Silver Plated Brass : Anklet : Beaded Anklets',
       'Fashion : Jewellery : Gold Plated Silver : Earrings : Drop Earrings',
       'Fashion : Jewellery : Silver Plated Brass : Ring : Statement Rings',
       'Fashion : Jewellery : Silver Plated Brass : Ring : Gemstone Rings',
       'Fashion : Jewellery : Costume : Necklace : Pendant Necklaces',
       'Fashion : Jewellery : Costume : Necklace : Long Necklaces',
       'Fashion : Jewellery : Costume : Necklace : Collar Necklaces',
       'Fashion : Jewellery : Costume : Earrings : Hoop Earrings',
       'Fashion : Jewellery : Costume : Earrings : Drop Earrings',
       'Fashion : Jewellery 

In [41]:
VERSION = "Oliver Bonas 2023 November Update"

levels = {
    0: "Departments",
    1: "Product Group",
    2: "Sub Product Group",
    3: "OB Type",
    4: "Type Detail"
}

In [42]:
external_ids_already_added = {""}

csv_dict_rows = []

for node in set(nodes):
    
    parent_chain = node.split(" : ")

    for i in range(len(parent_chain)):
        # parent is empty for fashion
        externalid = " : ".join(parent_chain[:i+1])
        parentid = " : ".join(parent_chain[:i])

        if externalid in external_ids_already_added:
            continue

        if parentid not in external_ids_already_added:
            raise Exception(f"parentid {parentid} not in external_ids_already_added")

        external_ids_already_added.add(externalid)

        csv_dict_rows.append({
            "ExternalID": externalid,
            "Name": parent_chain[i],
            "Description": "",
            "Version": VERSION,
            "level": levels[i],
            "Parentnode": parentid,
            "Is include": "TRUE",
        })

In [43]:
df = pandas.DataFrame(csv_dict_rows).sort_values(by="ExternalID")
df

,ExternalID,Name,Description,Version,level,Parentnode,Is include
0,Fashion,Fashion,,Oliver Bonas 2023 November Update,Departments,,TRUE
1,Fashion : Bags,Bags,,Oliver Bonas 2023 November Update,Product Group,Fashion,TRUE
75,Fashion : Bags : Backpack Bag,Backpack Bag,,Oliver Bonas 2023 November Update,Sub Product Group,Fashion : Bags,TRUE
253,Fashion : Bags : Backpack Bag : Draw String,Draw String,,Oliver Bonas 2023 November Update,OB Type,Fashion : Bags : Backpack Bag,TRUE
362,Fashion : Bags : Backpack Bag : Draw String : ...,Cotton,,Oliver Bonas 2023 November Update,Type Detail,Fashion : Bags : Backpack Bag : Draw String,TRUE
...,...,...,...,...,...,...,...
228,Home & Gift : Wellness : Sleep,Sleep,,Oliver Bonas 2023 November Update,Sub Product Group,Home & Gift : Wellness,TRUE
229,Home & Gift : Wellness : Sleep : Bedding,Bedding,,Oliver Bonas 2023 November Update,OB Type,Home & Gift : Wellness : Sleep,TRUE
809,Home & Gift : Wellness : Sleep : Eye Mask,Eye Mask,,Oliver Bonas 2023 November Update,OB Type,Home & Gift : Wellness : Sleep,TRUE
974,Home & Gift : Wellness : Sleep : Gift Set,Gift Set,,Oliver Bonas 2023 November Update,OB Type,Home & Gift : Wellness : Sleep,TRUE


In [44]:
df.to_csv("nnnooodddeeesss.csv", index=False)